In [288]:
import re
import pandas as pd
import numpy as np

In [289]:
# get transfer data
data = pd.read_csv("data/transfers1.8.csv")
data_len = len(data)
data.head()

,ID,arrival_club,arrival_league,player_name,age,position,transfer_price,year_1,year_2,origin_club,origin_league,grouping_positions_1,grouping_positions_2,arrival_club_tier,origin_club_tier,age_grouping_1,age_grouping_2,transfer_price_group1,transfer_price_group2
0,5427,Getafe CF,Primera Division,Derek Boateng,26,Defensive Midfield,0.675,2009,Before2016,1. FC Köln,1 Bundesliga,CDM,M,Tier_3,Tier_3,U30,Under30,Below1M,Below5M
1,4285,FC Schalke 04,1 Bundesliga,Peer Kluge,29,Defensive Midfield,1.350,2009,Before2016,1.FC Nuremberg,1 Bundesliga,CDM,M,Tier_2,Tier_4,U30,Under30,10Mto1M,Below5M
2,5453,Hertha BSC,1 Bundesliga,Artur Wichniarek,32,Centre Forward,0.630,2009,Before2016,Arm. Bielefeld,Other,CF,F,Tier_3,Tier_4,A30,Above30,Below1M,Below5M
3,4865,Hamburger SV,1 Bundesliga,Robert Tesche,22,Central Midfield,0.900,2009,Before2016,Arm. Bielefeld,Other,CM,M,Tier_2,Tier_4,U25,Under23,Below1M,Below5M
4,1361,Ajax Amsterdam,Eredivisie,Demy de Zeeuw,26,Central Midfield,7.200,2009,Before2016,AZ Alkmaar,Eredivisie,CM,M,Tier_2,Tier_3,U30,Under30,10Mto1M,20Mto5M


In [290]:
# get wikipedia player info
info_df = pd.read_csv("data/player_wikipedia_info.csv")
info_df.head()

,player_name,height,nationality
0,Neymar,1.75,Brazil
1,Philippe Coutinho,1.72,Brazil
2,Kylian Mbappé,1.78,France
3,João Félix,1.81,Portugal
4,Ousmane Dembélé,1.78,France


In [291]:
# get wikipedia player stats
stats_df = pd.read_csv("data/player_wikipedia_stats.csv")
stats_df.head()

,player_name,from_year,to_year,club,apps,gls
0,Neymar,2009,2013.0,Santos,177.0,107.0
1,Neymar,2013,2017.0,Barcelona,123.0,68.0
2,Neymar,2017,NaN,Paris Saint-Germain,52.0,47.0
3,Philippe Coutinho,2008,2013.0,Inter Milan,28.0,3.0
4,Philippe Coutinho,2008,2010.0,Vasco da Gama,19.0,1.0


In [292]:
# get fifa stats
fifa_df = pd.read_csv("data/fifa_stats.csv")
print(len(fifa_df))
fifa_df = fifa_df.applymap(lambda x: x.strip() if type(x)==str else x)
fifa_df.year = fifa_df.year.apply(lambda x: int("20" + str(x).zfill(2))) # convert two digit year to 4, and add lag of 1
fifa_df.drop_duplicates(subset=["player_name", "year"], inplace=True, keep=False)
fifa_df.reset_index(drop=True, inplace=True)
print(len(fifa_df))
fifa_df.head()

211042
202852


,ovr,year,nationality,player_name,pot
0,91,2008,Brazil,Ronaldinho,93
1,91,2008,Portugal,Cristiano Ronaldo,94
2,91,2008,Italy,Gianluigi Buffon,93
3,90,2008,England,Wayne Rooney,91
4,90,2008,England,John Terry,91


In [293]:
# merge potential ratings
data = data.merge(right=fifa_df[["player_name", "year", "pot", "nationality"]], left_on=["player_name", "year_1"], right_on=["player_name", "year"], how="left")
assert(len(data)==data_len)

# merge overall ratings, these are shifted by 1 year
ovr_fifa_df = fifa_df[["player_name", "ovr", "year"]]
ovr_fifa_df.year = ovr_fifa_df.year.apply(lambda x: x + 1)
data = data.merge(right=ovr_fifa_df, left_on=["player_name", "year"], right_on=["player_name", "year"], how="left")
data.drop(columns=["year"], inplace=True)
assert(len(data)==data_len)

data.head()

C:\Users\Shreyans\AppData\Local\Programs\Python\Python35\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,ID,arrival_club,arrival_league,player_name,age,position,transfer_price,year_1,year_2,origin_club,...,grouping_positions_2,arrival_club_tier,origin_club_tier,age_grouping_1,age_grouping_2,transfer_price_group1,transfer_price_group2,pot,nationality,ovr
0,5427,Getafe CF,Primera Division,Derek Boateng,26,Defensive Midfield,0.675,2009,Before2016,1. FC Köln,...,M,Tier_3,Tier_3,U30,Under30,Below1M,Below5M,NaN,NaN,NaN
1,4285,FC Schalke 04,1 Bundesliga,Peer Kluge,29,Defensive Midfield,1.350,2009,Before2016,1.FC Nuremberg,...,M,Tier_2,Tier_4,U30,Under30,10Mto1M,Below5M,84.0,Germany,77.0
2,5453,Hertha BSC,1 Bundesliga,Artur Wichniarek,32,Centre Forward,0.630,2009,Before2016,Arm. Bielefeld,...,F,Tier_3,Tier_4,A30,Above30,Below1M,Below5M,76.0,Poland,73.0
3,4865,Hamburger SV,1 Bundesliga,Robert Tesche,22,Central Midfield,0.900,2009,Before2016,Arm. Bielefeld,...,M,Tier_2,Tier_4,U25,Under23,Below1M,Below5M,74.0,Germany,64.0
4,1361,Ajax Amsterdam,Eredivisie,Demy de Zeeuw,26,Central Midfield,7.200,2009,Before2016,AZ Alkmaar,...,M,Tier_2,Tier_3,U30,Under30,10Mto1M,20Mto5M,78.0,Netherlands,71.0


In [294]:
# merge wikipedia height with dataset
data = data.merge(on="player_name", right=info_df[["player_name", "height"]])
assert(len(data)==data_len)
data.head()

,ID,arrival_club,arrival_league,player_name,age,position,transfer_price,year_1,year_2,origin_club,...,arrival_club_tier,origin_club_tier,age_grouping_1,age_grouping_2,transfer_price_group1,transfer_price_group2,pot,nationality,ovr,height
0,5427,Getafe CF,Primera Division,Derek Boateng,26,Defensive Midfield,0.675,2009,Before2016,1. FC Köln,...,Tier_3,Tier_3,U30,Under30,Below1M,Below5M,NaN,NaN,NaN,1.8288
1,4285,FC Schalke 04,1 Bundesliga,Peer Kluge,29,Defensive Midfield,1.350,2009,Before2016,1.FC Nuremberg,...,Tier_2,Tier_4,U30,Under30,10Mto1M,Below5M,84.0,Germany,77.0,1.7900
2,5453,Hertha BSC,1 Bundesliga,Artur Wichniarek,32,Centre Forward,0.630,2009,Before2016,Arm. Bielefeld,...,Tier_3,Tier_4,A30,Above30,Below1M,Below5M,76.0,Poland,73.0,1.8300
3,4865,Hamburger SV,1 Bundesliga,Robert Tesche,22,Central Midfield,0.900,2009,Before2016,Arm. Bielefeld,...,Tier_2,Tier_4,U25,Under23,Below1M,Below5M,74.0,Germany,64.0,1.8200
4,1361,Ajax Amsterdam,Eredivisie,Demy de Zeeuw,26,Central Midfield,7.200,2009,Before2016,AZ Alkmaar,...,Tier_2,Tier_3,U30,Under30,10Mto1M,20Mto5M,78.0,Netherlands,71.0,1.7400


In [295]:
# fill wikipedia nationality if NaN in dataset

data.nationality.fillna("", inplace=True)
for i in range(data_len):
    if data.nationality.iloc[i] == "":
        data.loc[i, "nationality"] = info_df[info_df.player_name == data.player_name.iloc[i]]["nationality"].iloc[0]
        
data.head()

,ID,arrival_club,arrival_league,player_name,age,position,transfer_price,year_1,year_2,origin_club,...,arrival_club_tier,origin_club_tier,age_grouping_1,age_grouping_2,transfer_price_group1,transfer_price_group2,pot,nationality,ovr,height
0,5427,Getafe CF,Primera Division,Derek Boateng,26,Defensive Midfield,0.675,2009,Before2016,1. FC Köln,...,Tier_3,Tier_3,U30,Under30,Below1M,Below5M,NaN,Ghana,NaN,1.8288
1,4285,FC Schalke 04,1 Bundesliga,Peer Kluge,29,Defensive Midfield,1.350,2009,Before2016,1.FC Nuremberg,...,Tier_2,Tier_4,U30,Under30,10Mto1M,Below5M,84.0,Germany,77.0,1.7900
2,5453,Hertha BSC,1 Bundesliga,Artur Wichniarek,32,Centre Forward,0.630,2009,Before2016,Arm. Bielefeld,...,Tier_3,Tier_4,A30,Above30,Below1M,Below5M,76.0,Poland,73.0,1.8300
3,4865,Hamburger SV,1 Bundesliga,Robert Tesche,22,Central Midfield,0.900,2009,Before2016,Arm. Bielefeld,...,Tier_2,Tier_4,U25,Under23,Below1M,Below5M,74.0,Germany,64.0,1.8200
4,1361,Ajax Amsterdam,Eredivisie,Demy de Zeeuw,26,Central Midfield,7.200,2009,Before2016,AZ Alkmaar,...,Tier_2,Tier_3,U30,Under30,10Mto1M,20Mto5M,78.0,Netherlands,71.0,1.7400


In [296]:
# merge goals/apps with dataset

stats_df.from_year = stats_df.from_year.str.extract(pat="(\d{4})")
stats_df.from_year = pd.to_numeric(stats_df.from_year, downcast='integer')

stats_df['overall_app'] = stats_df[['player_name','apps']].groupby(['player_name']).cumsum()
stats_df['overall_gls'] = stats_df[['player_name','gls']].groupby(['player_name']).cumsum()

stats_df.drop_duplicates(subset=["player_name", "to_year"], inplace=True)
stats_df.head()

,player_name,from_year,to_year,club,apps,gls,overall_app,overall_gls
0,Neymar,2009.0,2013.0,Santos,177.0,107.0,177.0,107.0
1,Neymar,2013.0,2017.0,Barcelona,123.0,68.0,300.0,175.0
2,Neymar,2017.0,NaN,Paris Saint-Germain,52.0,47.0,352.0,222.0
3,Philippe Coutinho,2008.0,2013.0,Inter Milan,28.0,3.0,28.0,3.0
4,Philippe Coutinho,2008.0,2010.0,Vasco da Gama,19.0,1.0,47.0,4.0


In [297]:
data = data.merge(right=stats_df[["player_name", "to_year", "overall_app", "overall_gls"]], how = 'left', left_on=["player_name", "year_1"], right_on=["player_name", 'to_year'])
assert(len(data)==data_len)

In [299]:
data.head()

,ID,arrival_club,arrival_league,player_name,age,position,transfer_price,year_1,year_2,origin_club,...,age_grouping_2,transfer_price_group1,transfer_price_group2,pot,nationality,ovr,height,to_year,overall_app,overall_gls
0,5427,Getafe CF,Primera Division,Derek Boateng,26,Defensive Midfield,0.675,2009,Before2016,1. FC Köln,...,Under30,Below1M,Below5M,NaN,Ghana,NaN,1.8288,2009.0,210.0,29.0
1,4285,FC Schalke 04,1 Bundesliga,Peer Kluge,29,Defensive Midfield,1.350,2009,Before2016,1.FC Nuremberg,...,Under30,10Mto1M,Below5M,84.0,Germany,77.0,1.7900,2009.0,250.0,19.0
2,5453,Hertha BSC,1 Bundesliga,Artur Wichniarek,32,Centre Forward,0.630,2009,Before2016,Arm. Bielefeld,...,Above30,Below1M,Below5M,76.0,Poland,73.0,1.8300,NaN,NaN,NaN
3,4865,Hamburger SV,1 Bundesliga,Robert Tesche,22,Central Midfield,0.900,2009,Before2016,Arm. Bielefeld,...,Under23,Below1M,Below5M,74.0,Germany,64.0,1.8200,2009.0,78.0,9.0
4,1361,Ajax Amsterdam,Eredivisie,Demy de Zeeuw,26,Central Midfield,7.200,2009,Before2016,AZ Alkmaar,...,Under30,10Mto1M,20Mto5M,78.0,Netherlands,71.0,1.7400,2009.0,184.0,22.0


In [301]:
data.to_csv("data/transfer_2.0.csv")